In [1]:
# import the necessary libraries 
import pandas as pd
import numpy as np
import seaborn as sns
from __future__ import division
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import gtf files in order to get the strand information 
# this is only necessary for the strand information 
gtf=pd.read_table('c_elegans.PRJNA13758.WS263.canonical_geneset.gtf',names=['chr','a','b','c','d','e','strand','g','gene'], comment='#')
# Get WB_ID from the GTF file; WB_ID =WormbaseID
WB_ID=[]
for i in gtf['gene']:
    i=i.split()[1]
    i=i.rstrip(";")
    i=i.split(("\""))[1]
    WB_ID.append(i)
gtf['WB_ID']=WB_ID 

In [ ]:
#UP_Stream_Gene

# we need two seperate gtfs to merged with the two intergenic regions to get the strand information this will be for 
# the Up_Stream_Gene

gtf1=gtf[['WB_ID','strand']]
gtf1=gtf1.drop_duplicates()
gtf1.rename(columns={'WB_ID':'Up_Stream_Gene'},inplace=True)

gtf1.reset_index(inplace=True,drop=True)
gtf1.head(1) # looks good 

In [ ]:
gtf2=gtf[['WB_ID','strand']]
gtf2=gtf2.drop_duplicates()
gtf2.rename(columns={'WB_ID':'Down_Stream_Gene'},inplace=True)

gtf2.reset_index(inplace=True,drop=True)

In [ ]:
# From wormbase this is the location of all of the intergenic regions in C. elegans
# we need to parse this out

df=pd.read_table('235_intergenic.txt',names=['location'])
df.head(1)

In [ ]:
# Get rid of the start of chromosome locations and the end of chromosome locations

special_start=[]
for i in df['location']:
    if "star" in i:
        special_start.append(i)
    
#special_start  

special_end=[]
for i in df['location']:
    if "end" in i:
        special_end.append(i)

#special_end #

In [ ]:
# get the length information from file 
length=[]
for i in df.location:
    i=i.split()[4]
    length.append(i)

df['length']=length
type(df['length'][1])
df.length = pd.to_numeric(df.length, errors='coerce') # the length had to be converted to a number since it was from a 
df.head(2)                                              # string

In [ ]:
! head -n 2 235_intergenic.txt # sanity check that length is equal to len from file (looks ok)

In [ ]:
# Parse the start location from the file 

start=[]
for i in df.location:
    i=i.split()[2]
    i=i.split(",")[0]
    start.append(i)
    
df['Start']=start

df.Start = pd.to_numeric(df.Start, errors='coerce')
df.head(2)

In [ ]:
! head -n 2 235_intergenic.txt # looks the same as the file 

In [ ]:
# Calculate the End of the intergenic location; should simply be Start + Length 

df['End']=df['Start'] + df['length']

In [ ]:
df.head(1)

In [ ]:
(df['End']-df['Start'])[:5] # sanity check

In [ ]:
df.head(2)

In [ ]:
# Get the upstreamGene, aka the gene that is the first one 
up_stream_gene=[]
for i in df['location']:
    i=i.split("_")[0]
    i=i.split(">")[1]
    up_stream_gene.append(i)
df['Up_Stream_Gene']=up_stream_gene

In [ ]:
# Get the upstreamGene, aka the gene that is the first one 
# same for downstream

down_stream_gene=[]
for i in df['location']:
    i=i.split("_")[1]
    i=i.split()[0]
    down_stream_gene.append(i)
df['Down_Stream_Gene']=down_stream_gene

In [ ]:
# Chromosome  
Chrom=[]
for i in df['location']:
    if "start" in i:
        Chrom.append(i)
    elif "end" in i:
        Chrom.append(i)
    else:
        i=i.split("_")[1]
        i=i.split()[1]
        Chrom.append(i)
df['Chr']=Chrom


In [ ]:
df_gene1=pd.merge(df,gtf1,how='inner',on='Up_Stream_Gene')

print(len(df_gene1))
print(len(df))

# what are the ones that are lost; maybe they are not real genes 

In [ ]:
df_gene1.head(1)

In [ ]:
df_gene1=df_gene1[~df_gene1['location'].str.contains("end")]

In [ ]:
df_gene2=pd.merge(df,gtf2,how='inner',on='Down_Stream_Gene')

# Making Upstream Gene SAF
SAF file is a file format that featureCounts can use to count reads 
Only need GeneID Start End Chr Strand

The goal is to get the 5' Junction for a Gene on the negative strand 
This gene will be the ustream gene since transcription will be going in the opposite direction 


GENE1_____________________GENE2
           <---
If gene 1 IS on the negative strand then the 5'junction will be the start location from the annotation file

To get a 5' junction go 10 nts in from the start(start-10), to get the end (in the intergenic region) add 10 nts

This will give a 21 nt region from which to get reads


In [ ]:
# Make the up_stream_gene_SAF
saf_up_stream_gene=df_gene1[['Up_Stream_Gene','Chr','Start','End','strand']]
saf_up_stream_gene=saf_up_stream_gene[saf_up_stream_gene.strand=='-']
saf_up_stream_gene.head(1)

In [ ]:
saf_up_stream_gene=saf_up_stream_gene.rename(columns={'Up_Stream_Gene':'GeneID',
                                                     'strand':'Strand'})



In [ ]:
saf_up_stream_gene.drop('End',inplace=True,axis=1)

In [ ]:
saf_up_stream_gene['End']=saf_up_stream_gene['Start'] + 10

In [ ]:
saf_up_stream_gene['New_Start']=saf_up_stream_gene['Start'] - 10

In [ ]:
saf_up_stream_gene.drop('Start',inplace=True,axis=1)

In [ ]:
saf_up_stream_gene=saf_up_stream_gene.rename(columns={'New_Start':'Start'})
saf_up_stream_gene=saf_up_stream_gene[['GeneID','Chr','Start','End','Strand']]

In [ ]:
saf_up_stream_gene.to_csv('saf_up_stream_Gene.saf',sep='\t',index=False)

In [ ]:
saf_up_stream_gene.tail(1) # make sure that it looks ok
                           # Randomly access rows and then go to the Genome Browser to verify that it worked 

# Make SAF for Down Stream Gene

Similar logic to above 

If we want to get the 5 prime junction for a gene on the plus strand we want to look at the downstream Gene 

GENE1___________________GENE2 
                     ----->  

In [ ]:
saf_down_stream_gene=df_gene2[['Down_Stream_Gene','Chr','Start','End','strand']]


In [ ]:
saf_down_stream_gene=saf_down_stream_gene[saf_down_stream_gene.strand=='+']

In [ ]:
saf_down_stream_gene=saf_down_stream_gene.rename(columns={'strand':'Strand',
                                  'Up_Stream_Gene':'GeneID'})



In [ ]:
saf_down_stream_gene.drop('Start',inplace=True,axis=1)

In [ ]:
saf_down_stream_gene['Start']=saf_down_stream_gene['End']-10

In [ ]:
saf_down_stream_gene['new_End']=saf_down_stream_gene['End']+10

In [ ]:
saf_down_stream_gene.drop('End',inplace=True,axis=1)

In [ ]:
saf_down_stream_gene=saf_down_stream_gene.rename(columns={'new_End':'End',
                                                         'Down_Stream_Gene':'GeneID'})


In [ ]:
saf_down_stream_gene=saf_down_stream_gene[['GeneID','Chr','Start','End','Strand']]

In [ ]:
saf_down_stream_gene.to_csv('saf_down_stream_Gene.saf',sep='\t',index=False)

In [ ]:
saf_down_stream_gene.head()